# Metric RAG

* median, decile, status (Red / Amber / Green), percentage
* All cost catgeories - cost per pupil
* Utilities and Premises and staff - per m^2

In [7]:
import numpy as np
import pandas as pd
import pickle
import comparator_sets as comparators
import glob 
import os

# Create and clean directory
from pathlib import Path
Path("output/metric-rag").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/metric-rag/*")
for f in files:
    os.remove(f)

## Loading saved comparator sets

In [8]:
with open('output/comparator-sets/schools.pkl', 'rb') as schools_file:
     all_schools = pickle.load(schools_file)

with open('output/comparator-sets/pupil_comparators.pkl', 'rb') as pupil_file:
     pupil_comparators = pickle.load(pupil_file)
     
with open('output/comparator-sets/building_comparators.pkl', 'rb') as building_file:
     building_comparators = pickle.load(building_file)

In [115]:
target_school = 103341
comparator_set = comparators.get_comparator_set_by(lambda s: s.index == target_school, all_schools, pupil_comparators).set_index('URN')

Declare the RAG mappings

In [129]:
category_settings = {
    'Teaching and Teaching support staff': {
        'type': 'pupil',
        'outstanding_10': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red'],
        'other_10': ['red', 'red', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'other': ['red', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red']
    },
    'Non-educational support staff': {
        'type': 'pupil',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    },
    'Educational supplies': {
        'type': 'pupil',
        'outstanding_10': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red'],
        'other_10': ['red', 'red', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'other': ['red', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red']
    },
    'IT': {
        'type': 'pupil',
        'outstanding_10': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red'],
        'other_10': ['red', 'red', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'other': ['red', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red']
    },
    'Premises': {
        'type': 'area',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    },
    'Utilities': {
        'type': 'area',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    },
    'Administrative supplies': {
        'type': 'pupil',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    },
    'Catering staff and supplies': {
        'type': 'pupil',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    },
    'Other costs': {
        'type': 'pupil',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    }
}


def is_area_close_comparator(y, x):
    return (((x['Total Internal Floor Area'] * 0.9) >= y['Total Internal Floor Area']) | (y['Total Internal Floor Area'] <= (x['Total Internal Floor Area'] * 1.1)) 
             & ((x['Age Average Score'] * 0.75) >= y['Age Average Score']) | (y['Age Average Score'] <= (x['Age Average Score'] * 1.25)))

def is_pupil_close_comparator(y, x):
    return (((x['Number of pupils'] * 0.75) >= y['Number of pupils']) | (y['Number of pupils'] <= (x['Number of pupils'] * 1.25)) 
             & ((x['Percentage Free school meals'] * 0.95) >= y['Percentage Free school meals']) | (y['Percentage Free school meals'] <= (x['Percentage Free school meals'] * 1.05))
             & ((x['Percentage SEN'] * 0.90) >= y['Percentage SEN']) | (y['Percentage SEN'] <= (x['Percentage SEN'] * 1.10))
            )

def is_close_comparator(t, y, x):
    if t == 'area': 
        return is_area_close_comparator(y, x)
    else: 
        return is_pupil_close_comparator(y, x)

def map_rag(d, ofstead, rag_mapping):
    close_count = d['is_close'][d['is_close']].count()
    key = 'outstanding' if ofstead.lower() == 'outstanding' else 'other'
    key += '_10' if close_count > 10 else ''
    d['rag'] = d['decile'].fillna(0).map(lambda x: rag_mapping[key][int(x)])
    return d    


Compute the rag calculation for the cost categories given in the [Benchmarking cost categories doc](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/DfEFinancialBenchmarking/_layouts/15/Doc.aspx?sourcedoc=%7B622FB0F9-7CB1-445A-8FFA-664F8857F036%7D&file=Benchmarking%20cost%20categories%20and%20sub-categories.docx&action=default&mobileredirect=true)

In [130]:
base_cols = ['URN', 'Total Internal Floor Area','Age Average Score', 'OfstedRating (name)','Percentage SEN', 'Percentage Free school meals', 'Number of pupils']

result = {}
for category in category_settings:
    deciles = category_settings[category]
    cols = (comparator_set.columns.isin(base_cols) | comparator_set.columns.str.startswith(category))
    
    df = comparator_set[comparator_set.columns[cols]].copy()
    target = comparator_set[comparator_set.index == target_school][comparator_set.columns[cols]].copy()
    
    if deciles['type'] == 'area':
        df['Total'] = df[df.columns[6:df.shape[1]]].sum(axis=1) / (df['Total Internal Floor Area'])
    else:
        df['Total'] = df[df.columns[6:df.shape[1]]].sum(axis=1) / (df['Number of pupils'])
        
    df['is_close'] = df.apply(lambda x: is_close_comparator(deciles['type'], target, x), axis=1)
    df['decile'] = pd.qcut(df['Total'], 10, labels=False, duplicates='drop')
    
    result[category] = map_rag(df, target['OfstedRating (name)'].values[0], deciles).sort_values(by='decile', ascending=True)
    
for key in result:
    display(result[key])


,Number of pupils,OfstedRating (name),Percentage Free school meals,Percentage SEN,Total Internal Floor Area,Age Average Score,Teaching and Teaching support staff_Agency supply teaching staff,Teaching and Teaching support staff_Education support staff,Teaching and Teaching support staff_Educational consultancy,Teaching and Teaching support staff_Supply teaching staff,Teaching and Teaching support staff_Teaching staff,Total,is_close,decile,rag
URN,,,,,,,,,,,,,,,
145110,417.0,Requires improvement,18.7,5.164319,3163.000000,51.245969,185.00,1204.00,106.00,1.00,7275.00,21.033573,True,0,amber
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,NaN,NaN,NaN,0.000000,True,0,amber
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,NaN,NaN,NaN,0.000000,True,0,amber
142295,382.0,Good,15.6,4.838710,3730.000000,63.332440,425.00,2208.00,229.00,0.00,6748.00,25.157068,True,1,amber
139634,373.0,Good,19.9,4.199475,5483.691451,53.585434,396.00,3099.00,324.00,0.00,11098.00,39.991957,True,1,amber
142044,345.0,Good,17.8,5.219780,2987.000000,53.703047,103.00,3781.00,313.00,0.00,8652.00,37.243478,True,1,amber
139674,409.0,Outstanding,17.3,3.855422,3676.000000,63.177911,3211.00,23276.00,729.00,0.00,48891.00,186.080685,False,2,amber
141601,333.0,Outstanding,17.9,5.555556,1966.000000,75.499491,602.00,8904.00,653.00,266.00,21178.00,94.903904,True,2,amber
145824,383.0,Good,16.8,5.208333,2775.000000,69.956036,0.00,18173.00,1310.00,2767.00,61523.00,218.728460,True,2,amber


,Number of pupils,OfstedRating (name),Percentage Free school meals,Percentage SEN,Total Internal Floor Area,Age Average Score,Non-educational support staff_Administrative and clerical staff,Non-educational support staff_Auditor costs,Non-educational support staff_Other staff,Non-educational support staff_Professional services (non-curriculum),Total,is_close,decile,rag
URN,,,,,,,,,,,,,,
145110,417.0,Requires improvement,18.7,5.164319,3163.000000,51.245969,763.00,19.0,95.00,91.00,2.321343,True,0,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,NaN,NaN,0.000000,True,0,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,NaN,NaN,0.000000,True,0,green
142295,382.0,Good,15.6,4.838710,3730.000000,63.332440,616.00,29.0,217.00,159.00,2.672775,True,1,green
139634,373.0,Good,19.9,4.199475,5483.691451,53.585434,1334.00,35.0,822.00,124.00,6.206434,True,1,green
142044,345.0,Good,17.8,5.219780,2987.000000,53.703047,1371.00,29.0,0.00,777.00,6.310145,True,1,green
139674,409.0,Outstanding,17.3,3.855422,3676.000000,63.177911,12987.00,97.0,2087.00,251.00,37.706601,False,2,green
141601,333.0,Outstanding,17.9,5.555556,1966.000000,75.499491,3716.00,46.0,954.00,372.00,15.279279,True,2,green
145824,383.0,Good,16.8,5.208333,2775.000000,69.956036,9614.00,137.0,3679.00,110.00,35.352480,True,2,green


,Number of pupils,OfstedRating (name),Percentage Free school meals,Percentage SEN,Total Internal Floor Area,Age Average Score,Educational supplies_Examination fees,Educational supplies_Learning resources (not ICT equipment),Total,is_close,decile,rag
URN,,,,,,,,,,,,
145110,417.0,Requires improvement,18.7,5.164319,3163.000000,51.245969,125.0,261.00,0.925659,True,0,amber
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,0.000000,True,0,amber
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,0.000000,True,0,amber
142295,382.0,Good,15.6,4.838710,3730.000000,63.332440,0.0,561.00,1.468586,True,1,amber
139634,373.0,Good,19.9,4.199475,5483.691451,53.585434,26.0,452.00,1.281501,True,1,amber
142044,345.0,Good,17.8,5.219780,2987.000000,53.703047,0.0,729.00,2.113043,True,1,amber
139674,409.0,Outstanding,17.3,3.855422,3676.000000,63.177911,0.0,5697.00,13.929095,False,2,amber
141601,333.0,Outstanding,17.9,5.555556,1966.000000,75.499491,0.0,1271.00,3.816817,True,2,amber
145824,383.0,Good,16.8,5.208333,2775.000000,69.956036,836.0,6046.00,17.968668,True,2,amber


,Number of pupils,OfstedRating (name),Percentage Free school meals,Percentage SEN,Total Internal Floor Area,Age Average Score,IT_ICT learning resources,Total,is_close,decile,rag
URN,,,,,,,,,,,
103341,380.0,Outstanding,18.2,5.263158,4835.000000,59.699483,0.00,0.000000,True,0,amber
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,0.000000,True,0,amber
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,0.000000,True,0,amber
142295,382.0,Good,15.6,4.838710,3730.000000,63.332440,71.00,0.185864,True,1,amber
139634,373.0,Good,19.9,4.199475,5483.691451,53.585434,103.00,0.276139,True,1,amber
145110,417.0,Requires improvement,18.7,5.164319,3163.000000,51.245969,96.00,0.230216,True,1,amber
141601,333.0,Outstanding,17.9,5.555556,1966.000000,75.499491,118.00,0.354354,True,2,amber
113081,344.0,Good,16.6,4.651163,1117.000000,6.385855,113.13,0.328866,False,2,amber
145824,383.0,Good,16.8,5.208333,2775.000000,69.956036,206.00,0.537859,True,2,amber


,Number of pupils,OfstedRating (name),Percentage Free school meals,Percentage SEN,Total Internal Floor Area,Age Average Score,Premises_Cleaning and caretaking,Premises_Maintenance of premises,Premises_Other occupation costs,Premises_Premises staff,Total,is_close,decile,rag
URN,,,,,,,,,,,,,,
145110,417.0,Requires improvement,18.7,5.164319,3163.000000,51.245969,229.00,0.00,91.00,177.00,0.157129,True,0,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,NaN,NaN,0.000000,True,0,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,NaN,NaN,0.000000,True,0,green
142295,382.0,Good,15.6,4.838710,3730.000000,63.332440,188.00,115.00,81.00,274.00,0.176408,True,1,green
139634,373.0,Good,19.9,4.199475,5483.691451,53.585434,242.00,184.00,112.00,589.00,0.205518,True,1,green
142044,345.0,Good,17.8,5.219780,2987.000000,53.703047,576.00,1181.00,3.00,146.00,0.638098,True,1,green
141601,333.0,Outstanding,17.9,5.555556,1966.000000,75.499491,871.00,605.00,203.00,754.00,1.237538,True,2,green
132251,404.0,Good,19.6,5.198020,2403.000000,17.000000,0.00,0.00,1956.74,0.00,0.814290,False,2,green
145824,383.0,Good,16.8,5.208333,2775.000000,69.956036,1608.00,1105.00,1120.00,1943.00,2.081441,True,2,green


,Number of pupils,OfstedRating (name),Percentage Free school meals,Percentage SEN,Total Internal Floor Area,Age Average Score,Utilities_Energy,Utilities_Water and sewerage,Utilities_Water and sewerage:,Total,is_close,decile,rag
URN,,,,,,,,,,,,,
145110,417.0,Requires improvement,18.7,5.164319,3163.000000,51.245969,109.00,21.0,NaN,0.041100,True,0,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,NaN,0.000000,True,0,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,NaN,0.000000,True,0,green
142295,382.0,Good,15.6,4.838710,3730.000000,63.332440,179.00,54.0,NaN,0.062466,True,1,green
139634,373.0,Good,19.9,4.199475,5483.691451,53.585434,349.00,57.0,NaN,0.074038,True,1,green
142044,345.0,Good,17.8,5.219780,2987.000000,53.703047,226.00,0.0,NaN,0.075661,True,1,green
139674,409.0,Outstanding,17.3,3.855422,3676.000000,63.177911,1388.00,351.0,NaN,0.473069,True,2,green
141601,333.0,Outstanding,17.9,5.555556,1966.000000,75.499491,610.00,135.0,NaN,0.378942,True,2,green
145824,383.0,Good,16.8,5.208333,2775.000000,69.956036,1522.00,390.0,NaN,0.689009,True,2,green


,Number of pupils,OfstedRating (name),Percentage Free school meals,Percentage SEN,Total Internal Floor Area,Age Average Score,Administrative supplies_Administrative supplies (non educational),Total,is_close,decile,rag
URN,,,,,,,,,,,
145824,383.0,Good,16.8,5.208333,2775.000000,69.956036,0.00,0.000000,True,0,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,0.000000,True,0,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,0.000000,True,0,green
145110,417.0,Requires improvement,18.7,5.164319,3163.000000,51.245969,389.00,0.932854,True,1,green
142295,382.0,Good,15.6,4.838710,3730.000000,63.332440,355.00,0.929319,True,1,green
142044,345.0,Good,17.8,5.219780,2987.000000,53.703047,227.00,0.657971,True,1,green
139674,409.0,Outstanding,17.3,3.855422,3676.000000,63.177911,1961.00,4.794621,False,2,green
139634,373.0,Good,19.9,4.199475,5483.691451,53.585434,569.00,1.525469,True,2,green
141601,333.0,Outstanding,17.9,5.555556,1966.000000,75.499491,1620.00,4.864865,True,2,green


,Number of pupils,OfstedRating (name),Percentage Free school meals,Percentage SEN,Total Internal Floor Area,Age Average Score,Total,is_close,decile,rag
URN,,,,,,,,,,
103341,380.0,Outstanding,18.2,5.263158,4835.000000,59.699483,0.0,True,NaN,green
120583,381.0,Good,18.6,4.461942,1843.000000,36.267499,0.0,False,NaN,green
145824,383.0,Good,16.8,5.208333,2775.000000,69.956036,0.0,True,NaN,green
117156,403.0,Requires improvement,18.4,4.962779,2330.000000,122.216738,0.0,True,NaN,green
101262,358.0,Good,19.3,5.865922,3165.000000,66.617694,0.0,True,NaN,green
115853,405.0,Good,17.5,4.444444,2826.000000,77.000000,0.0,False,NaN,green
132251,404.0,Good,19.6,5.198020,2403.000000,17.000000,0.0,True,NaN,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,0.0,True,NaN,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,0.0,True,NaN,green


,Number of pupils,OfstedRating (name),Percentage Free school meals,Percentage SEN,Total Internal Floor Area,Age Average Score,Other costs_Direct revenue financing,Other costs_Grounds maintenance,Other costs_Indirect employee expenses,Other costs_Interest charges for loan and bank,...,Other costs_PFI charges,Other costs_Rent and rates,Other costs_Special facilities,Other costs_Staff development and training,Other costs_Staff-related insurance,Other costs_Supply teacher insurance,Total,is_close,decile,rag
URN,,,,,,,,,,,,,,,,,,,,,
142044,345.0,Good,17.8,5.219780,2987.000000,53.703047,-22.00,60.00,0.00,0.00,...,0.00,70.00,0.00,55.00,0.00,0.00,0.472464,True,0,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True,0,green
142036,406.0,Good,16.9,5.494505,2503.000000,49.601678,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True,0,green
145110,417.0,Requires improvement,18.7,5.164319,3163.000000,51.245969,0.00,11.00,8.00,0.00,...,0.00,137.00,1.00,17.00,0.00,0.00,0.505995,True,1,green
142295,382.0,Good,15.6,4.838710,3730.000000,63.332440,22.00,28.00,33.00,0.00,...,0.00,50.00,0.00,74.00,0.00,0.00,0.693717,True,1,green
139634,373.0,Good,19.9,4.199475,5483.691451,53.585434,216.00,58.00,104.00,0.00,...,0.00,97.00,37.00,102.00,0.00,0.00,1.823056,True,1,green
139674,409.0,Outstanding,17.3,3.855422,3676.000000,63.177911,1205.00,256.00,462.00,83.00,...,0.00,748.00,0.00,487.00,178.00,0.00,9.283619,False,2,green
141601,333.0,Outstanding,17.9,5.555556,1966.000000,75.499491,0.00,147.00,114.00,0.00,...,1.00,226.00,0.00,139.00,0.00,207.00,2.990991,True,2,green
145824,383.0,Good,16.8,5.208333,2775.000000,69.956036,1061.00,198.00,325.00,0.00,...,425.00,1578.00,0.00,843.00,6.00,0.00,11.582245,True,2,green
